In [15]:
from z3 import *

In [16]:
# Function to read the .dat file and return the parameters
def read_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse m (number of couriers) and n (number of items)
    m = int(lines[0].strip())
    n = int(lines[1].strip())

    # Parse load limits
    load_limits = list(map(int, lines[2].strip().split()))

    # Parse item sizes
    item_sizes = list(map(int, lines[3].strip().split()))

    # Parse the distance matrix
    distance_matrix = []
    for i in range(n + 1):  # n+1 because the last row is the origin
        distance_matrix.append(list(map(int, lines[4 + i].strip().split())))

    return m, n, load_limits, item_sizes, distance_matrix


In [17]:
# Function to solve the MCP problem using Z3 Optimize
def solve_mcp(file_path):
    # Read instance data from .dat file
    m, n, load_limits, item_sizes, distance_matrix = read_instance(file_path)

    # Z3 optimizer initialization
    optimizer = Optimize()

    # Boolean variables: x[i][j] = True if courier i is assigned item j
    x = [[Bool(f"x_{i}_{j}") for j in range(n)] for i in range(m)]

    # Variables for the total distance traveled by each courier
    total_distance = [Int(f"D_{i}") for i in range(m)]

    # Maximum distance traveled by any courier (to minimize)
    max_distance = Int('Z')

    # 1. Each item must be assigned to exactly one courier
    for j in range(n):
        optimizer.add(Sum([If(x[i][j], 1, 0) for i in range(m)]) == 1)

    # 2. The load carried by each courier must not exceed its capacity
    for i in range(m):
        optimizer.add(Sum([If(x[i][j], item_sizes[j], 0) for j in range(n)]) <= load_limits[i])

    # 3. Distance calculation for each courier based on item assignments
    for i in range(m):
        courier_distances = []
        for j in range(n):
            for k in range(n):
                if j != k:
                    courier_distances.append(If(And(x[i][j], x[i][k]), distance_matrix[j][k], 0))

        # Distance from the origin to the first assigned item and back to origin
        origin_distances = [If(x[i][j], distance_matrix[n][j] + distance_matrix[j][n], 0) for j in range(n)]

        optimizer.add(total_distance[i] == Sum(courier_distances) + Sum(origin_distances))

    # 4. Maximize the minimum distance traveled by any courier
    # Adding constraints that max_distance is >= total distance for each courier
    for i in range(m):
        optimizer.add(max_distance >= total_distance[i])

    # Objective: Minimize the maximum distance
    optimizer.minimize(max_distance)

    # Solve the problem
    if optimizer.check() == sat:
        model = optimizer.model()
        print(f"Minimized max distance: {model[max_distance]}")
        
        for i in range(m):
            assigned_items = [j for j in range(n) if model[x[i][j]]]
            print(f"Courier {i+1} is assigned items: {assigned_items} with distance {model[total_distance[i]]}")
    else:
        print("No solution found")


In [8]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst01.dat')

Minimized max distance: 51
Courier 1 is assigned items: [2, 3, 5] with distance 51
Courier 2 is assigned items: [0, 1, 4] with distance 48


In [9]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst03.dat')

Minimized max distance: 42
Courier 1 is assigned items: [1, 3, 4] with distance 42
Courier 2 is assigned items: [2, 6] with distance 24
Courier 3 is assigned items: [0, 5] with distance 24


In [10]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst04.dat')

Minimized max distance: 220
Courier 1 is assigned items: [0] with distance 106
Courier 2 is assigned items: [8] with distance 34
Courier 3 is assigned items: [6] with distance 126
Courier 4 is assigned items: [9] with distance 80
Courier 5 is assigned items: [2, 4] with distance 220
Courier 6 is assigned items: [5] with distance 150
Courier 7 is assigned items: [1, 3] with distance 219
Courier 8 is assigned items: [7] with distance 220


In [11]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst05.dat')

Minimized max distance: 458
Courier 1 is assigned items: [1] with distance 160
Courier 2 is assigned items: [0, 2] with distance 458


In [12]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst06.dat')

Minimized max distance: 322
Courier 1 is assigned items: [4] with distance 308
Courier 2 is assigned items: [7] with distance 322
Courier 3 is assigned items: [3] with distance 202
Courier 4 is assigned items: [0] with distance 226
Courier 5 is assigned items: [1, 6] with distance 302
Courier 6 is assigned items: [2, 5] with distance 120


In [13]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst07.dat')

No solution found


In [14]:
def solve_mcp_const(file_path):
    # Read instance data from .dat file
    m, n, load_limits, item_sizes, distance_matrix = read_instance(file_path)

    # Z3 optimizer initialization
    optimizer = Optimize()

    # Boolean variables: Z[i][j] is True if courier i is assigned item j
    Z = [[Int(f"Z_{i}_{j}") for j in range(n+1)] for i in range(m)]  # m couriers, n items + origin

    # Variables for the total distance traveled by each courier
    Dist = [Int(f"D_{i}") for i in range(m)]

    # Variables to track which courier carries which item
    Carry = [Int(f"Carry_{j}") for j in range(n)]

    # Define upper bound UB (e.g., a large number for the distances)
    UB = sum([sum(row) for row in distance_matrix])

    # Lower bound LB: Minimum possible distance (e.g., farthest node)
    LB = max([max(row) for row in distance_matrix])

    # 1. Lower Bound Constraint
    optimizer.add(max(Dist) >= LB)

    # 2. Subcircuit Constraint (Ensuring each route is a valid permutation)
    for i in range(m):
        optimizer.add(Distinct(Z[i]))  # Each row is a distinct permutation (like the subcircuit constraint)

    # 3. Load capacities must be respected
    for i in range(m):
        optimizer.add(Sum([If(Z[i][j] != j + 1, item_sizes[j], 0) for j in range(n)]) <= load_limits[i])

    # 4. Each item can only be carried by one courier
    for j in range(n):
        optimizer.add(Sum([If(Z[i][j] != j + 1, 1, 0) for i in range(m)]) == 1)

    # 5. Each courier must leave the depot
    for i in range(m):
        optimizer.add(Z[i][n] != n + 1)  # Z[i,n+1] != n+1

    # 6. Distance formulation for each courier
    for i in range(m):
        courier_distances = []
        for j in range(n):
            courier_distances.append(If(Z[i][j] != j + 1, distance_matrix[j][Z[i][j]], 0))

        origin_distances = [If(Z[i][j] != j + 1, distance_matrix[n][j] + distance_matrix[j][n], 0) for j in range(n)]

        optimizer.add(Dist[i] == Sum(courier_distances) + Sum(origin_distances))

    # 7. Carry assignment formulation
    for j in range(n):
        for i in range(m):
            optimizer.add(Implies(Z[i][j] != j + 1, Carry[j] == i + 1))

    # Redundant constraint: Enforcing load capacities (bin-packing problem)
    optimizer.add(Sum([If(Carry[j] == i + 1, item_sizes[j], 0) for j in range(n)]) <= load_limits[i] for i in range(m))

    # Objective: Minimize the maximum distance
    optimizer.minimize(max(Dist))

    # Solve the problem
    if optimizer.check() == sat:
        model = optimizer.model()
        print(f"Minimized max distance: {model[max(Dist)]}")
        
        for i in range(m):
            assigned_items = [j for j in range(n) if model[Z[i][j]] != j + 1]
            print(f"Courier {i+1} is assigned items: {assigned_items} with distance {model[Dist[i]]}")
    else:
        print("No solution found")

In [18]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp('/Users/shariqansari/Documents/CDMO/instances/inst01.dat')

Minimized max distance: 51
Courier 1 is assigned items: [2, 3, 5] with distance 51
Courier 2 is assigned items: [0, 1, 4] with distance 48


In [19]:
# Call the solve_mcp function with the path to the .dat instance file
solve_mcp_const('/Users/shariqansari/Documents/CDMO/instances/inst01.dat')

Z3Exception: Symbolic expressions cannot be cast to concrete Boolean values.

In [26]:
def solve_mcp_hard(file_path):
    # Read instance data from .dat file
    m, n, load_limits, item_sizes, distance_matrix = read_instance(file_path)

    # Z3 optimizer initialization
    optimizer = Optimize()

    # Integer variables: Z[i][j] represents the destination node courier i goes after visiting node j
    Z = [[Int(f"Z_{i}_{j}") for j in range(n+1)] for i in range(m)]  # m couriers, n items + origin

    # Variables for the total distance traveled by each courier
    Dist = [Int(f"D_{i}") for i in range(m)]

    # Variables to track which courier carries which item
    Carry = [Int(f"Carry_{j}") for j in range(n)]

    # Define upper bound UB (e.g., a large number for the distances)
    UB = sum([sum(row) for row in distance_matrix])

    # Lower bound LB: Minimum possible distance (e.g., farthest node)
    LB = max([max(row) for row in distance_matrix])

    # 1. Lower Bound Constraint (Ensure the maximum distance is greater than or equal to LB)
    max_dist = Dist[0]
    for i in range(1, m):
        max_dist = If(Dist[i] > max_dist, Dist[i], max_dist)  # Z3's version of `max`
    optimizer.add(max_dist >= LB)

    # 2. Subcircuit Constraint (Ensuring each route is a valid permutation)
    for i in range(m):
        optimizer.add(Distinct(Z[i]))  # Each row is a distinct permutation (like the subcircuit constraint)

    # 3. Load capacities must be respected
    for i in range(m):
        optimizer.add(Sum([If(Z[i][j] != j + 1, item_sizes[j], 0) for j in range(n)]) <= load_limits[i])

    # 4. Each item can only be carried by one courier
    for j in range(n):
        optimizer.add(Sum([If(Z[i][j] != j + 1, 1, 0) for i in range(m)]) == 1)

    # 5. Each courier must leave the depot
    for i in range(m):
        optimizer.add(Z[i][n] != n + 1)  # Z[i,n+1] != n+1

    # 6. Distance formulation for each courier
    for i in range(m):
        courier_distances = []
        for j in range(n):
            # Use conditional If statements to compute distance based on Z[i][j]
            courier_distances.append(
                Sum([If(Z[i][j] == k, distance_matrix[j][k], 0) for k in range(n+1)])
            )

        # Distance from the origin to the first assigned item and back to origin
        origin_distances = [If(Z[i][j] != j + 1, distance_matrix[n][j] + distance_matrix[j][n], 0) for j in range(n)]

        optimizer.add(Dist[i] == Sum(courier_distances) + Sum(origin_distances))

    # 7. Carry assignment formulation
    for j in range(n):
        for i in range(m):
            optimizer.add(Implies(Z[i][j] != j + 1, Carry[j] == i + 1))

    # 8. Enforcing load capacities (bin-packing problem) per courier
    for i in range(m):
        optimizer.add(Sum([If(Carry[j] == i + 1, item_sizes[j], 0) for j in range(n)]) <= load_limits[i])

    # Objective: Minimize the maximum distance
    optimizer.minimize(max_dist)
    max_dist = Int('max_dist') 

    # Solve the problem
    if optimizer.check() == sat:
        model = optimizer.model()
        print(f"Minimized max distance: {model[max_dist]}")
        
        for i in range(m):
            assigned_items = [j for j in range(n) if model[Z[i][j]] != j + 1]
            print(f"Courier {i+1} is assigned items: {assigned_items} with distance {model[Dist[i]]}")
    else:
        print("No solution found")



In [27]:
solve_mcp_hard('/Users/shariqansari/Documents/CDMO/instances/inst01.dat')

Minimized max distance: None


Z3Exception: Symbolic expressions cannot be cast to concrete Boolean values.